# Tools  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
Tools allow agents to 'Act' in the real world. 
Careful descriptions can help your agent discover how to use your tools.

LangChain supports many tool formats and tool sets. Here we will cover some common cases, but check the [docs](https://docs.langchain.com/oss/python/langchain/tools) for more information.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****hPcA
LANGSMITH_API_KEY=****a745
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


## Calculator example

In this example, the docstring and inferred arguments and argument types are used by the LLM to detetermine when and how to call the tool.

In [2]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers."""
    print("🧮 Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [3]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
my_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    max_tokens=None,
    temperature=0 # Lower temperature for stable, factual SQL generation
)#new
agent = create_agent(
    #model="openai:gpt-5"
    model=my_llm,
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

This invokes your calculator tool.

In [4]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


🧮 Invoking calculator tool


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


The product of 3.1125 and 4.1234 is 12.8340825.


In [5]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 ** 3"}]}
)
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


[{'type': 'text', 'text': 'I can only perform addition, subtraction, multiplication, and division. I cannot calculate exponents.', 'extras': {'signature': 'Cp8DAdHtim+0Od2nQyIA3cKX2x2PirQUUlFYmv7263ChLXidbaY5wdB14ZYWKn8iYJpQueywHwkkmw3ltDsIMy7a9tLTAza6cws/N/ELL1Ga7jBMYLLJsWTZrifSvxOJiKj5Dj/2zYsx/CUw7nC/LZgMqp7hQqhQyY9TGGYY10FHpV0czzjKUMnNxTLtrxBKoU0jhpDVKQXqyBJ/dgnfgKAYNNc5LV9wYKwah2jo7PMPs3K05CWy1Zw//v8T/Az2LayLY7b9dP8fyDzOgg7hUNUiO6Ht+a9SwhCoj+qaqwAdkp7LuFlUZzxIfgUMbUXpLZyYFSlCJ6PBvuQdvqjQms0jtdPdpsYp8YSE+yOqQL+aI5KncTDKG9q4xqzdpDwdWxQbcDUk88sBG0z01HzjBg7tdDC46Xcy/C9YDiaptRt8pNd6Xqvj+Rie8OHpWwhVREZYXqn/EyrQhAnK2VCSVgMOPzKt2lG9RkzP84sJdxTtS0k2IsFVa1rzO3/XAdX0xFKd+ilJCtLsFlsOX5mClD+11mmelqyjOfOdTs5isq/7KQ=='}}]


We can check the **metadata** in [LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r) to see this.

The tool description can have a big impact. 
This may not  invoke your calculator tool because the inputs are integers.  (results vary from run to run)

In [6]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


🧮 Invoking calculator tool
3 * 4 = 12


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


This often does not invoke the tool though the input are real numbers. (results vary from run to run)

In [7]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


🧮 Invoking calculator tool


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


3.0 * 4.0 = 12


## Adding a more detailed description
While a basic description is often sufficient, LangChain has support for enhanced descriptions. The example below uses one method: Google Style argument descriptions. Used with `parse_docstring=True`, this will parse and pass the arg descriptions to the model. You can rename the tool and change its description. This can be effective when you are sharing a standard tool but would like agent-specific instructions.

In [8]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.
            - `"multiply"`: Returns the product of `a` and `b`.
            - `"divide"`: Returns the result of `a / b`. Raises an error if `b` is zero.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("🧮  Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [9]:
from langchain.agents import create_agent

agent = create_agent(
    #model="openai:gpt-5-mini"
    model=my_llm,
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [10]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


🧮  Invoking calculator tool
3.0 * 4.0 = 12


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Let's check our [LangSmith Observability trace](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r) to see the tool description.

In [11]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


🧮  Invoking calculator tool


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


3 * 4 = 12


## Try your own.
Create a tool of your own and try it here!

In [27]:
import math
from typing import Literal
from langchain.tools import tool


# --- Activation function implementations (you were missing these) ---

def relu(x: float) -> float:
    """ReLU activation function."""
    return max(0.0, x)

def sigmoid(x: float) -> float:
    """Sigmoid activation function."""
    return 1 / (1 + math.exp(-x))


# --- LangChain tool ---

@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Apply activation functions (ReLU or sigmoid) to two real numbers "
        "and return the sum of both results. "
        "Use this whenever activation processing is required."
    ),
)
def activation(
    a: float,
    b: float,
    operation: Literal["relu", "sigmoid"]
) -> float:
    """Apply an activation function (ReLU or sigmoid) to two inputs and return the sum.

    Args:
        a (float): First numeric input.
        b (float): Second numeric input.
        operation (Literal["relu", "sigmoid"]):
            The activation function to apply.

            - `"relu"`: Returns relu(a) + relu(b)
            - `"sigmoid"`: Returns sigmoid(a) + sigmoid(b)

    Returns:
        float: The sum of the activation outputs.

    Raises:
        ValueError: If an invalid operation is provided.
    """
    
    print("🧮 Invoking activation calculator tool")

    if operation == "relu":
        return relu(a) + relu(b)
    elif operation == "sigmoid":
        return sigmoid(a) + sigmoid(b)
    else:
        raise ValueError(f"Invalid operation: {operation}")


In [28]:
from langchain.agents import create_agent

agent = create_agent(
    #model="openai:gpt-5-mini"
    model=my_llm,
    tools=[activation],
    system_prompt="You are a helpful assistant",
)

In [30]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is sigmoid of 3 and sigmoid of 4"}]}
)
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


🧮 Invoking activation calculator tool


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


The sum of the sigmoid of 3 and the sigmoid of 4 is 1.9345879168603419.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
